# Init

In [1]:
import tensorflow as tf
import os
import pathlib
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from pathlib import Path
from PIL import Image
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder

from keras.preprocessing.image import load_img,img_to_array,array_to_img



# Tutorial

In [2]:

r1=tf.constant(1,tf.int8)

In [ ]:
r1

In [ ]:
display(r1)

In [ ]:
r1.shape

In [ ]:
tf.shape(r1)


In [ ]:
r1.ndim 

In [ ]:
r2=tf.constant(1,tf.int16,name="my_scalar")

In [ ]:
r2

In [ ]:
r1_decimal=tf.constant(1.12345,tf.float32)

In [ ]:
r1_decimal

In [ ]:
print(r1_decimal)

In [ ]:
print(r2)

In [ ]:
r1_vector = tf.constant([1,3,5], tf.int16)

In [ ]:
r1_vector.shape

In [ ]:
r1_vector.ndim

In [ ]:
r2_matrix=tf.constant([[1,2],[3,4]],tf.int16)

In [ ]:
r3_matrix=tf.constant([ [[1, 2],
                           [3, 4], 
                           [5, 6]] ], tf.int16)
print(r3_matrix)

In [ ]:
r10=tf.ones([10, 10])
print(r10)

In [ ]:
x=tf.constant([2.0],dtype=tf.float32)
print(tf.sqrt(x))

# 26/7/2021

In [ ]:
tf.__version__

In [ ]:
PATH_TO_LABELS = '/home/rchuzh/programming/image_labelling_shrdc/src/lib/machine_learning/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

In [ ]:
DATA_DIR = '/home/chuzh/Documents/TensorFlow/workspace/training_demo'
MODELS_DIR = '/home/rchuzh/programming/image_labelling_shrdc/resources/dl_models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'

PATH_TO_CKPT = os.path.join(MODELS_DIR, 'checkpoint/')
PATH_TO_CFG = os.path.join(MODELS_DIR, 'pipeline.config')

LABEL_FILENAME = 'labelmap.pbtxt'
PATH_TO_LABELS = os.path.join(MODELS_DIR, LABEL_FILENAME)


In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

In [ ]:
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
configs

In [ ]:
model_config = configs['model']
model_config

In [ ]:
detection_model = model_builder.build(
        model_config=model_config, is_training=False)
detection_model


In [ ]:
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()
ckpt

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                        use_display_name=True)

category_index

In [ ]:
IMAGE_PATH='/home/rchuzh/Desktop/chair.jpg'
image=load_img(IMAGE_PATH)
display(type(image),image)


In [ ]:
image=img_to_array(image)
display(image,image.shape)
image_expanded=np.expand_dims(image, axis=0)
image_expanded.shape

In [ ]:
input_tensor = tf.convert_to_tensor(
    image_expanded, dtype=tf.float32)
type(input_tensor)

In [ ]:
@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

In [ ]:
detections, predictions_dict, shapes = detect_fn(input_tensor)

In [ ]:
detections

In [ ]:
label_id_offset = 1
image_np_with_detections = image.copy()

In [ ]:
vis_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'][0].numpy(),
            (detections['detection_classes']
             [0].numpy() + label_id_offset).astype(int),
            detections['detection_scores'][0].numpy(),
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.10,
            agnostic_mode=False)
image_np_with_detections=array_to_img(image_np_with_detections)



In [ ]:
image_np_with_detections

# 13/8/2021

In [14]:
import tensorflow as tf
import time
from datetime import datetime
import timeit
tf.__version__
display("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2021-08-14 00:01:24.103102: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


'The following GPU devices are available: /device:GPU:0'

2021-08-14 00:01:24.104004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-14 00:01:24.104860: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-14 00:01:24.105740: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-14 00:01:24.106664: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-14 00:01:24.107413: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 1

In [4]:
@tf.function
def function(x):
    a=tf.constant([[2.0],[3.0]])
    b=tf.constant(4.0)
    return a+b

In [7]:
# Plot a graph for function() using Tensorboard
stamp=datetime.now().strftime("%Y%m%d-%H%M%S")
logdir='logs/func/%s'%stamp
writer=tf.summary.create_file_writer(logdir)
tf.summary.trace_on(graph=True,profiler=True)
#Call only one tf.function when tracing
z=function(2)
with writer.as_default():
    tf.summary.trace_export(
        name='function_trace',
        step=0,
        profiler_outdir=logdir
    )

2021-08-13 23:53:56.628438: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-08-13 23:53:56.628488: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-08-13 23:53:56.820201: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-13 23:53:56.820944: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-08-13 23:53:56.892857: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 3 callback api events and 2 activity events. 
2021-08-13 23:53:56.893851: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [ ]:
%%bash
%load_ext tensorboard
%tensorboard --logdir logs/func

In [13]:
def callable_func(a,b):
    return tf.matmul(a,b)

#create a Function object that contains a graph
function_that_uses_graph=tf.function(callable_func)

#Make some tensors
a1=tf.constant([[1.0,2.0]])
b1=tf.constant([[2.0],[3.0]])

function_that_uses_graph(a1,b1).numpy()

array([[8.]], dtype=float32)

In [ ]:
class SequentialModel(tf.keras.Model):
  def __init__(self, **kwargs):
    super(SequentialModel, self).__init__(**kwargs)
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.dense_1 = tf.keras.layers.Dense(128, activation="relu")
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)

  def call(self, x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return x

input_data = tf.random.uniform([60, 28, 28])

eager_model = SequentialModel()
graph_model = tf.function(eager_model)

print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=10000))
print("Graph time:", timeit.timeit(lambda: graph_model(input_data), number=10000))